In [87]:
"""Общий анализ опросов

Картинки должны печататься на уровне соответствующего subname
"""

import os
import re
import openpyxl
from openpyxl import Workbook
from matplotlib import pyplot as plt
from openpyxl import load_workbook
import numpy as np
import pandas as pd
import math as math
%matplotlib inline

poll_number=5 #tmp file indicator

files =['C:/Users/d.koshman/Downloads/data/Автомобили_2017-07-06_14-35-56.xlsx',\
        'C:/Users/d.koshman/Downloads/data/Дом_и_семья_2017-07-06_14-18-13.xlsx',\
       'C:/Users/d.koshman/Downloads/data/Здоровье_2017-07-06_14-43-23.xlsx',\
       'C:/Users/d.koshman/Downloads/data/Путешествия_2017-07-06_14-48-36.xlsx',\
       'C:/Users/d.koshman/Downloads/data/Работа_и_учеба_2017-07-06_14-28-02.xlsx',\
       'C:/Users/d.koshman/Downloads/data/Товары_и_услуги_2017-07-06_14-54-12.xlsx']
file = pd.read_excel(files[poll_number],header=None)

In [88]:
poll_names=[]
for i in range(len(files)):
    tmp=re.split(r'\/',files[i])[-1]
    pattern= re.compile(r'(.*?).xlsx')
    poll_names.append(pattern.search(tmp).groups()[0])

In [89]:
file=file.iloc[:-2]
pure = file.iloc[4:]
pure.columns=['id','sex','age','country','city','region'] + list(file.iloc[3][6:])

In [90]:
text=file.iloc[:4, 6:]
text.columns=list(range(text.shape[1]))
ekrani = np.array(text.iloc[0].get_values())
ekrani = np.array([str(i) for i in ekrani])
ekrani = np.array([i for i in ekrani if i != 'nan'])

In [91]:
class Block:
    """A block for analysing specific values"""
    def __init__(self):
        self.height = 6
        self.image= []
        self.description = None
        self.name =None
        self.subname=[]
        self.df = []
    
    def clear(self):
        self.height = 6
        self.image= []
        self.description = None
        self.name =None
        self.subname=[]
        self.df = []

In [92]:
import re
class X:
    """holds global values"""
    total=pure.shape[0]
    males=pure[pure['sex']=='m'].shape[0]
    females=pure[pure['sex']=='f'].shape[0]
    

class Ques:
    def __init__(self, name, answers=[]):
        self.name=name
        self.answers=answers
    
pattern= re.compile(r'S\[(\d*)\]Q\[(\d*)\]A\[(\d*)\]')
krani=[]
for i in range(text.shape[1]):
    location=pattern.search(text.iloc[3,i]).groups()
    
    if int(location[0]) > len(krani):
        krani.append([])
        krani[-1].append(Ques(text.iloc[1,i],[text.iloc[2,i]]))
    
    elif int(location[1]) >len(krani[-1]):
        krani[-1].append(Ques(text.iloc[1,i],[text.iloc[2,i]]))
        
    elif True:
        krani[-1][-1].answers.append(text.iloc[2,i])
        

In [93]:
"""content creation"""
x=X()
sheets=[]
sheets.append([])

sheet_number=0 #to be placed like this: sheets[sheet_number][i]

sheets[0].append(Block())
sheets[0][0].name=poll_names[poll_number]

sheets[0][0].subname.append('sex')
data=[]
data.append(['Мужчин','Женщин','unknown'])
data.append(["{0:.2f}%".format(x.males/x.total * 100), "{0:.2f}%".format(x.females/x.total * 100), "{0:.2f}%".format((x.total-x.males-x.females)/x.total * 100)])
data.append([x.males, x.females, x.total-x.males-x.females])
df=pd.DataFrame(data)
sheets[0][0].df.append(df)

sheets[0][0].subname.append('age')
clean_ages = pure[pure['age']<100]['age']
data=[]
data.append(['average', 'median', 'std dev'])
data.append([ '{0:.2f}'.format(clean_ages.mean()), '{0:.2f}'.format(clean_ages.median()), '{0:.2f}'.format(clean_ages.std()) ])
df=pd.DataFrame(data)
plt.hist(clean_ages, bins=200, label='age distribution')
plt.ylabel('distibution')
plt.xlabel('ages')
image_name=str(1)+'_image.png'
try:
    os.remove(image_name)
except:
    pass
plt.savefig(image_name)
sheets[0][0].image.append(image_name)
plt.close()
sheets[0][0].df.append(df)

sheets[0][0].subname.append('По странам:')
data = pd.DataFrame(pure['country'].value_counts()[:20])
data['0']=list(data.T)
data = data.T.sort_index()
sheets[0][0].df.append(data)
sheets[0][0].subname.append('По регионам:')
data=pd.DataFrame(pure['region'].value_counts()[:20])
data['0'] = [name.split(' ')[0][:10] for name in list(data.T)]
data = data.T.sort_index()
sheets[0][0].df.append(data)
sheets[0][0].subname.append('По городам:')
data=pd.DataFrame(pure['city'].value_counts()[:20])
data['0'] = [name.split(' ')[0][:10] for name in list(data.T)]
data = data.T.sort_index()
sheets[0][0].df.append(data)

for i in range(len(ekrani)):
    sheets[0].append(Block())
    sheets[0][len(sheets)].name=ekrani[i]
    
    for j in range(len(krani[i])):
        sheets[0][len(sheets)].subname.append(krani[i][j].name)
        data=[[], [], []]
        yeah_sum=0
        
        for k in range(len(krani[i][j].answers)):
            data[2].append(krani[i][j].answers[k])
            yeah = pure[pure['S[{}]Q[{}]A[{}]'.format(i+1,j+1,k+1)]=='True'].shape[0]
            nope = pure[pure['S[{}]Q[{}]A[{}]'.format(i+1,j+1,k+1)]=='False'].shape[0]
            if (yeah+nope) !=0:
                data[1].append('{0:.2f}%'.format(100*yeah/(yeah+nope)))
            else:
                data[1].append('{0:.2f}%'.format(0))
            data[0].append(yeah)
            yeah_sum+=yeah
            
        if yeah_sum==0:
            data=pd.DataFrame(pure['S[{}]Q[{}]A[{}]'.format(i+1,j+1,k+1)].value_counts()[:20])
            data1=pd.DataFrame(pure['S[{}]Q[{}]A[{}]'.format(i+1,j+1,k+1)].value_counts()[:20])
            data1.columns=['1']
            for t in range(data1.shape[0]):
                data1.iloc[t] ='{0:.2f}%'.format(100*data.get_value(data.index[t], data.columns[0])  /x.total)
            data2= data.merge(data1, left_index=True, right_index=True )
            data2['0'] = [name for name in list(data2.T)]
            data2=data2.T
            data2 = data2.sort_index()
            data2['0'] = ['Ответ:','Частота:','Количество:']
            qwe= [data2.columns[-1]]+list(data2.columns[:-1])
            data2 = data2[qwe]
            

        if yeah_sum >0:
            sheets[0][len(sheets)].df.append(pd.DataFrame(data).T)
        else:
            sheets[0][len(sheets)].df.append(pd.DataFrame(data2))
    

In [94]:
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment

try: 
    wb =  load_workbook(poll_names[poll_number]+ '_analysis'+'.xlsx')
except:
    wb= Workbook()
ws=wb.worksheets[0]
"""clear a sheet"""
wb.remove_sheet(ws)
ws=wb.create_sheet()

big_cells=list('DEFGHIJKLMNOPQRSTUVW')
ws.column_dimensions['A'].width=5
for letter in big_cells:
    ws.column_dimensions[letter].width = 12

hpadding=2
curr_row=2
curr_ws=0
curr_col =1
image_lowest_row=0
subnames_lowest_row=0



for i in range(len(sheets)):
    for j in range(len(sheets[i])):
        """print block"""
        
        curr_block=sheets[i][j]
        curr_col=hpadding
        
        ws.cell(column=curr_col, row=curr_row).value=curr_block.name
        curr_row+=1
        curr_col+=1
        
        if (len(curr_block.df)>0) & (len(curr_block.subname) == 0):
            rows = curr_block.df[0].values.tolist()
            df_col=curr_col
            for row in rows:
                for cell in row:
                    ws.cell(column=curr_col, row=curr_row).value =cell
                    curr_col+=1
                curr_col=df_col
                curr_row+=1
            curr_row+=1
        
        
        curr_col+=1
        for k in range(len(curr_block.subname)):
            """print subnames + dataframes"""
            
            ws.cell(column=curr_col-1, row=curr_row).value=curr_block.subname[k]
            curr_row+=1
            df_col=curr_col
    
            
            
            if len(curr_block.df[k:])>0:
                rows = curr_block.df[k].values.tolist()
                for row in rows:
                    for cell in row:
                        ws.cell(column=curr_col, row=curr_row).value =cell
                        ws.cell(column=curr_col, row=curr_row).alignment = Alignment(horizontal='left')
                        curr_col+=1
                    curr_col=df_col
                    curr_row+=1
            curr_row+=1
        
        subnames_lowest_row=curr_row   
        
        for m in range(len(curr_block.image)):
            """draw stuff"""
            img = openpyxl.drawing.image.Image(curr_block.image[m])
            img.anchor(ws.cell(column=curr_col, row=curr_row))
            ws.add_image(img)
            curr_row+=14
            image_lowest_row = curr_row
                    

        if subnames_lowest_row >= image_lowest_row:
            curr_row=subnames_lowest_row+ 1
        else:
            curr_row=image_lowest_row+1
        
wb.save(poll_names[poll_number]+ '_analysis'+'.xlsx')